In [1]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir("../..")
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/soutrik-vm-dev/code/Users/Soutrik.Chowdhury/langgraph_ventures


In [3]:
from src.query_router import get_question_router_chain
from src.doc_grader import get_doc_grader_chain
from src.query_rewriter import get_query_rewrite_chain
from src.utils import get_llm, get_embedder
from src.doc_summarizer import get_doc_summarizer_chain
from src.response_scorer import get_response_scorer_chain

In [4]:
query = "What are the potential benefits of investing in the financials sector under the new Republican administration"
content = """
Nutmeg Investor Update: December 2024 – January 2025

Introduction
The end of 2024 saw strong performance in global equity and bond markets, with US stocks leading gains. The re-election of Donald Trump and a Republican sweep in the US election boosted investor optimism for US assets, while emerging markets and Europe faced headwinds.

Market Highlights

US Equities: Large-cap US stocks rose 5.9% and small caps 10.2% in November. The S&P 500 gained 25% for 2024, and the NASDAQ nearly 26%, driven by robust corporate earnings and optimism over tax breaks and deregulation.
Global Markets: Japanese and UK equities posted modest gains, while European markets declined amid concerns over US trade tariffs and weak industrial activity. Emerging markets fell 3.5% in November, with China underperforming but India remaining resilient.
Bonds: Both government and corporate bonds rose in November. High-yield bonds in the US and Europe gained over 8% for the year, reflecting investor risk appetite. However, government bonds had mixed results, with UK gilts underperforming and US bonds muted due to fewer expected rate cuts.
Outlook for 2025
Nutmeg’s investment team remains optimistic but cautious. The US economic environment is supportive, with a strong job market and declining inflation. However, high US equity valuations and geopolitical risks, including potential Trump administration policies and global conflicts, could increase volatility. Emerging markets face challenges from a stronger dollar and possible tariffs. The team is holding higher cash reserves to respond to market opportunities and risks.

Conclusion
Despite uncertainties, Nutmeg maintains a positive long-term outlook, emphasizing diversification and readiness to adapt to changing market conditions.

The Nutmeg investment team is closely monitoring the evolving landscape and is prepared to adjust strategies as necessary. The focus remains on delivering value to clients through informed investment decisions and a commitment to long-term growth.
"""
source = "Source idx: 0 - https://www.nutmeg.com/nutmegonomics/nutmeg-investor-update-december-2024\n\nSource idx: 1 - https://www.nutmeg.com/nutmegonomics/nutmeg-investor-update-march-2025\n\nSource idx: 2 - https://www.nutmeg.com/nutmegonomics/nutmeg-investor-update-january-2025\n\nSource idx: 3 - https://www.nutmeg.com/nutmegonomics/nutmeg-investor-update-december-2024\n\nSource idx: 4 - https://www.nutmeg.com/nutmegonomics/nutmeg-investor-update-february-2025\n\nSource idx: 5 - https://www.nutmeg.com/nutmegonomics/nutmeg-investor-update-january-2025"
keywords = "Keywords set: 0 - (Nutmeg investor update December 2024 US election Donald Trump equity markets emerging markets trade tariffs global trade US stocks economic growth bond markets investment implications),\nKeywords set: 1 - (Nutmeg investor update March 2025 US stocks European equities Chinese equities geopolitical news US macroeconomic signals consumer confidence financial markets investment outlook portfolio changes financial sector interest rates Republican administration),\nKeywords set: 2 - (Nutmeg investor update January 2025 stock markets bonds Federal Reserve interest rates inflation economic growth Santa Claus rally S&P 500 NASDAQ AI corporate earnings geopolitical uncertainties market volatility US equity emerging markets China Europe UK),\nKeywords set: 3 - (Nutmeg investor update December 2024 US election Donald Trump equity markets emerging markets trade tariffs global trade US stocks economic growth bond markets investment implications),\nKeywords set: 4 - (Nutmeg investor update February 2025 global equities Donald Trump White House DeepSeek AI Bank of England rate cut bond market currency volatility),\nKeywords set: 5 - (Nutmeg investor update January 2025 stock markets bonds Federal Reserve interest rates inflation economic growth Santa Claus rally S&P 500 NASDAQ AI corporate earnings geopolitical uncertainties market volatility US equity emerging markets China Europe UK)"

In [5]:
llm = get_llm()
embedder = get_embedder()

##### ROUTER CHAIN

In [6]:
question_router_chain = get_question_router_chain(llm)

In [7]:
op = await question_router_chain.ainvoke(
    {
        "query": query,
    }
)
print(op.source)
op = await question_router_chain.ainvoke({"query": "What is the capital of France?"})
print(op.source)

vector_retriever
web_search


##### DOC GRADER

In [8]:
doc_grader_chain = get_doc_grader_chain(llm)

In [9]:
op = await doc_grader_chain.ainvoke(
    {
        "context": content,
        "question": query,
    }
)

print(op.grade_score)

no_relevance


##### Query Rewriter

In [10]:
query_rewrite_chain = get_query_rewrite_chain(llm)

In [11]:
op = query_rewrite_chain.invoke(
    {
        "context": keywords,
        "question": query,
    }
)
print(op)

rewritten_query='What are the potential benefits of investing in the financials sector under the new Republican administration, considering the recent US election results, economic growth, interest rates, and geopolitical news?'


##### DOC Summarizer

In [12]:
summarizer_chain = get_doc_summarizer_chain(llm)

In [13]:
op = summarizer_chain.invoke(
    {
        "question": query,
        "context": content,
        "citation": source,
    }
)
print(op)

summary="**Introduction**\n\nThe re-election of Donald Trump and a Republican sweep in the US election have created a favorable environment for the financials sector. The new administration's policies are expected to benefit this sector significantly.\n\n**Potential Benefits of Investing in the Financials Sector**\n\n- **Tax Breaks and Deregulation**: The Republican administration is likely to implement tax breaks and deregulation, which can boost corporate earnings and profitability in the financials sector.\n- **Investor Optimism**: The positive sentiment among investors towards US assets, driven by the new administration's policies, can lead to increased investments and higher valuations in the financials sector.\n- **Strong Economic Environment**: A supportive US economic environment, characterized by a strong job market and declining inflation, can further enhance the performance of financial stocks.\n- **High-Yield Bonds**: The rise in high-yield bonds reflects investor risk appe

In [14]:
response = op

##### Response Scorer

In [15]:
scorer_chain = get_response_scorer_chain(llm)

In [16]:
op = scorer_chain.invoke(
    {
        "response": response,
        "question": query,
        "context": content,
    }
)
print(op.hallucination)
print(op.response_score)

no
0.9


########## END ##########